# Local Search

In [ ]:
from typing import List
import random
import math
from functools import cached_property

In [ ]:
class LocalState:
    def __init__(self):
        pass

    @cached_property
    # @cached_property makes this property computed only once 
    # and cached for future access
    def objective(self) -> int | float:
        raise NotImplementedError("Subclasses should implement this method")
    
    def neighbors(self) -> List['LocalState']:
        raise NotImplementedError("Subclasses should implement this method")

    def __repr__(self) -> str:
        return "LocalState"

    def __hash__(self) -> int:
        return hash(self.__repr__())

## 8-Queen Problem: Complete-State Formulation

In [ ]:
class EightQueenLocalState(LocalState):
    def __init__(self, queens=None):
        if queens is None:
            self.queens = [i for i in range(1, 9)]
            random.shuffle(self.queens)
        else:
            self.queens = queens
        self.cached_obj_value = None

    def __eq__(self, other: 'EightQueenLocalState') -> bool:
        return self.queens == other.queens

    def __repr__(self):
        board = [['.' for _ in range(8)] for _ in range(8)]
        for col, row in enumerate(self.queens):
            board[row-1][col] = 'Q'
        return '\n'.join([' '.join(row) for row in board])
    
    def __hash__(self):
        return hash(tuple(self.queens))
    
    @cached_property
    # count the number of pairs of queens that attack each other
    def objective(self) -> int:
        attacks = 0
        for col1 in range(8):
            for col2 in range(col1 + 1, 8):
                row1, row2 = self.queens[col1], self.queens[col2]
                if row1 == row2 or abs(col1 - col2) == abs(row1 - row2):
                    attacks += 1
        return attacks

    # generate all possible neighbor states 
    # by moving each queen to a different row
    def neighbors(self) -> List['EightQueenLocalState']:
        neighbors = []
        for col in range(8):
            for row in range(1, 9):
                if row != self.queens[col]:
                    new_queens = self.queens[:]
                    new_queens[col] = row
                    neighbors.append(EightQueenLocalState(new_queens))
        return neighbors

## Hill-Climbing Search

In [ ]:
def hill_climbing(initial_state: LocalState) -> LocalState:
    current_state = initial_state
    while True:
        # generate all neighbors of current_state
        neighbors = current_state.neighbors()
        if not neighbors:
            # if there are no neighbors, we've reached a local optimum
            return current_state

        # next state candidate = the neighbor with min objective value
        next_state = min(neighbors, key=lambda state: state.objective)
        if next_state.objective >= current_state.objective:
            # if next state candidate is not better than current state
            return current_state
        
        current_state = next_state

In [5]:
initial_state = EightQueenLocalState([1, 2, 3, 4, 5, 6, 7, 8])
print("Initial State:")
print(initial_state)
solution = hill_climbing(initial_state)
print("Solution State:")
print(solution)
print("Objective Value:", solution.objective)

Initial State:
Q . . . . . . .
. Q . . . . . .
. . Q . . . . .
. . . Q . . . .
. . . . Q . . .
. . . . . Q . .
. . . . . . Q .
. . . . . . . Q
Solution State:
. Q Q . . . . .
. . . . . . . .
. . . . . Q . .
. . . . . . . Q
Q . . . . . . .
. . . Q . . . .
. . . . . . Q .
. . . . Q . . .
Objective Value: 1


## Random-Restart Hill Climbing Search

In [ ]:
def random_restart_hill_climbing(max_restarts: int = 10) -> LocalState:
    best_solution = None
    for _ in range(max_restarts):
        # generate a random initial state
        initial_state = EightQueenLocalState()

        # run hill climbing from the initial state
        current_solution = hill_climbing(initial_state)
        
        if best_solution is None or \
           current_solution.objective < best_solution.objective:
            # update best_solution if current_solution is better
            best_solution = current_solution
            
    return best_solution

In [7]:
solution = random_restart_hill_climbing(max_restarts=10)
print("Solution State:")
print(solution)
print("Objective Value:", solution.objective)

Solution State:
. . . . Q . . .
. . Q . . . . .
. . . . . . . Q
. . . Q . . . .
. . . . . . Q .
Q . . . . . . .
. . . . . Q . .
. Q . . . . . .
Objective Value: 0


## Simulated Annealing

In [ ]:
def simulated_annealing(initial_state: LocalState, 
                        max_iterations: int = 1000, 
                        initial_temp: float = 100.0, 
                        cooling_rate: float = 0.99) -> LocalState:
    current_state = initial_state
    temperature = initial_temp

    # repeat until max_iterations is reached
    for _ in range(max_iterations):
        # if temperature drops to 0, stop the process
        if temperature <= 0:
            break

        # generate all neighbors of current_state
        neighbors = current_state.neighbors()
        if not neighbors:
            # if there are no neighbors, we've reached a local optimum
            return current_state

        # next state candidate = a random neighbor
        next_state = random.choice(neighbors)
        # calculate the change in objective value
        delta = next_state.objective - current_state.objective
        
        if delta < 0 or \
           random.random() < (math.exp(-delta / temperature)):
            # if the change in objective value is better
            # or if we accept a worse solution with a certain probability,
            # move to the next state candidate
            # otherwise, stay in the current state
            current_state = next_state

        # update the temperature
        temperature *= cooling_rate
    
    return current_state

In [15]:
initial_state = EightQueenLocalState([1, 2, 3, 4, 5, 6, 7, 8])
print("Initial State:")
print(initial_state)
solution = simulated_annealing(initial_state)
print("Solution State:")
print(solution)
print("Objective Value:", solution.objective)

Initial State:
Q . . . . . . .
. Q . . . . . .
. . Q . . . . .
. . . Q . . . .
. . . . Q . . .
. . . . . Q . .
. . . . . . Q .
. . . . . . . Q
Solution State:
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . . . . Q
. . . . . Q . .
. . . Q . . . .
. Q . . . . . .
. . . . Q . . .
Objective Value: 0


## Genetic Algorithm

In [ ]:
def genetic_algorithm_8queens(population_size: int = 100, 
                              generations: int = 1000, 
                              mutation_rate: float = 0.01) -> LocalState:
    def crossover(parent1: EightQueenLocalState, 
                  parent2: EightQueenLocalState) -> EightQueenLocalState:
        point = random.randint(0, 7)
        new_queens = parent1.queens[:point] + parent2.queens[point:]
        return EightQueenLocalState(new_queens)

    def mutate(individual: EightQueenLocalState) -> EightQueenLocalState:
        if random.random() < mutation_rate:
            col = random.randint(0, 7)
            row = random.randint(1, 8)
            new_queens = individual.queens[:]
            new_queens[col] = row
            return EightQueenLocalState(new_queens)
        return individual
    
    def roulette_wheel_selection(population: List[EightQueenLocalState]) \
                                 -> EightQueenLocalState:
        # calculate the selection probabilities
        selection_weights = [28 - ind.objective for ind in population]
        return random.choices(population, weights=selection_weights, k=2)

    # generate initial population
    population = [EightQueenLocalState() for _ in range(population_size)]
    # select the best solution from the current population
    best_solution = min(population, key=lambda ind: ind.objective)

    for _ in range(generations):
        # order individuals by their objective value
        population.sort(key=lambda ind: ind.objective)
        # select the best individuals to carry forward
        next_generation = population[:population_size // 2]

        while len(next_generation) < population_size:
            # select two parents
            parent1, parent2 = roulette_wheel_selection(population)
            # perform crossover and mutation
            child = crossover(parent1, parent2)
            child = mutate(child)
            # add the child to the next generation
            next_generation.append(child)

        # move to the next generation
        population = next_generation

        # select the best individual from the current generation
        best_in_generation = min(population, key=lambda ind: ind.objective)
        if best_in_generation.objective < best_solution.objective:
            # update best_solution if the new one is better
            best_solution = best_in_generation

    return best_solution

In [ ]:
solution = genetic_algorithm_8queens(population_size=100, 
                                     generations=500, 
                                     mutation_rate=0.01)
print("Solution State:")
print(solution)
print("Objective Value:", solution.objective)

Solution State:
. . Q . . . . .
. . . . Q . . .
. . . . . . Q .
Q . . . . . . .
. . . Q . . . .
. Q . . . . . .
. . . . . . . Q
. . . . . Q . .
Objective Value: 0
